In [1]:
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.5/20.5 MB 56.4 MB/s  0:00:006m0:00:01


In [8]:
%pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 54.5 MB/s  0:00:006m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]2m1/2 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [12]:
import duckdb
import os

conn = duckdb.connect("quickbite_express.db")

dataset_path = "/workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets"

for file in os.listdir(dataset_path):
    if file.endswith(".csv"):
        table_name = file.replace(".csv", "")
        file_path = os.path.join(dataset_path, file)

        conn.execute(f"""
            CREATE OR REPLACE TABLE {table_name} AS
            SELECT * FROM '{file_path}';
        """)
        print(f"{table_name} table created using {file_path}")

fact_order_items table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/fact_order_items.csv
dim_customer table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/dim_customer.csv
dim_restaurant table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/dim_restaurant.csv
fact_ratings table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/fact_ratings.csv
fact_orders table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/fact_orders.csv
dim_menu_item table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/dim_menu_item.csv
fact_delivery_performance table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/fact_delivery_performance.csv
dim_delivery_partner_ table created using /workspaces/Crisis-Recovery-Data-Analytics-Project/Datasets/dim_delivery_partner_.csv


In [13]:
conn.execute("SHOW TABLES;").fetch_df()

,name
0,dim_customer
1,dim_delivery_partner_
2,dim_menu_item
3,dim_restaurant
4,fact_delivery_performance
5,fact_order_items
6,fact_orders
7,fact_ratings


In [15]:
conn.execute("SELECT * FROM fact_orders LIMIT 3;").fetch_df()

,order_id,customer_id,restaurant_id,delivery_partner_id,order_timestamp,subtotal_amount,discount_amount,delivery_fee,total_amount,is_cod,is_cancelled
0,ORD202501023439,CUST181110,REST08622,DP05541,2025-01-01 12:00:00,471.62,35.44,30.56,466.74,N,N
1,ORD202501012051,CUST025572,REST02383,DP08091,2025-01-01 12:00:00,255.68,0.00,27.45,283.13,Y,N
2,ORD202501019281,CUST179306,REST14069,DP02021,2025-01-01 12:00:00,428.38,0.00,26.23,454.61,N,N


In [16]:
conn.execute("SELECT * FROM fact_order_items LIMIT 3;").fetch_df()

,order_id,item_id,menu_item_id,restaurant_id,quantity,unit_price,item_discount,line_total
0,ORD202501006518,ITEM001,MENU12962_3216,REST12962,2,48.31,0.0,96.62
1,ORD202501006518,ITEM002,MENU12962_1962,REST12962,3,61.24,0.0,183.71
2,ORD202501019281,ITEM001,MENU14069_0510,REST14069,2,87.19,0.0,174.38


In [17]:
conn.execute("SELECT * FROM fact_ratings LIMIT 3;").fetch_df()

,order_id,customer_id,restaurant_id,rating,review_text,review_timestamp,sentiment_score
0,ORD202501023439,CUST181110,REST08622,4.5,Super fast delivery,01-01-2025 15:00,0.75
1,ORD202501019281,CUST179306,REST14069,4.5,Great taste!,01-01-2025 15:00,0.75
2,ORD202501018036,CUST093042,REST13907,5.0,Super fast delivery,01-01-2025 14:03,1.00


In [18]:
conn.execute("SELECT * FROM fact_delivery_performance LIMIT 3;").fetch_df()

,order_id,actual_delivery_time_mins,expected_delivery_time_mins,distance_km
0,ORD202501023439,31,31,6.4
1,ORD202501012051,46,42,1.9
2,ORD202501019281,25,31,6.2


In [19]:
conn.execute("SELECT * FROM dim_customer LIMIT 3;").fetch_df()

,customer_id,signup_date,city,acquisition_channel
0,CUST000007,2025-03-21,Pune,Organic
1,CUST000008,2025-02-07,Kolkata,Referral
2,CUST000009,2025-02-25,Delhi,Paid


In [21]:
conn.execute("SELECT * FROM dim_delivery_partner_ LIMIT 3;").fetch_df()

,delivery_partner_id,partner_name,city,vehicle_type,employment_type,avg_rating,is_active
0,DP09615,Neha E,Bengaluru,Scooter,Full-time,3.77,Y
1,DP02021,Neha V,Bengaluru,Bike,Full-time,3.90,Y
2,DP05541,Vikas S,Delhi,Cycle,Part-time,3.33,Y


In [22]:
conn.execute("SELECT * FROM dim_menu_item LIMIT 3;").fetch_df()

,menu_item_id,restaurant_id,item_name,category,is_veg,price
0,MENU12962_3216,REST12962,Paneer Tikka Pizza,Pizza,Y,271.05
1,MENU12962_1962,REST12962,Pepperoni Pizza,Pizza,N,327.26
2,MENU14069_0510,REST14069,Egg Fried Rice,Fried Rice,N,200.01


In [23]:
conn.execute("SELECT * FROM dim_restaurant LIMIT 3;").fetch_df()

,restaurant_id,restaurant_name,city,cuisine_type,partner_type,avg_prep_time_min,is_active
0,REST12962,Flavours of Sweets Palace,Bengaluru,Chinese,Restaurant,26-40,N
1,REST14069,Royal Biryani Darbar,Ahmedabad,Fast Food,Restaurant,16-25,Y
2,REST08622,Spicy Wraps Point,Mumbai,North Indian,Restaurant,16-25,Y
